In [3]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import potrace
import os

In [ ]:
# FUNCTIONS
def detectFaceEdges(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY) # convert to grayscale for processing

    # FACE DETECTION ______________________
    # Get the directory of the current script
    script_dir = os.path.dirname(os.path.realpath('TESTS.ipynb'))
    cascade_path = os.path.join(script_dir, 'haarcascade_frontalface_default.xml')

    # Load Haar Cascade from the same directory as the script
    face_cascade = cv.CascadeClassifier(cascade_path)
    if face_cascade.empty():
        raise ValueError("Haar cascade file could not be loaded.")

    # Face detection
    faces = face_cascade.detectMultiScale(img, 1.3, 5)

    if len(faces) == 0:
        print("No Face Detected!")
        return None, None
    elif len(faces) > 3:
        print("More than 3 Faces Detected!")
        return None, None        

    # Get Region of Interest
    roi = img.copy()
    for (x,y,w,h) in faces:
        roi = roi[y:y+h, x:x+w]
    
    roi = cv.resize(roi, (250, 250)) # resized to a standard size for canvas creation 
    
    # EDGE DETECTION ______________________
    # Add Gussian Blur and Detect Edges
    img_blur = cv.GaussianBlur(roi, (3,3), 0)
    edge_img = cv.Canny(image=img_blur, threshold1=50, threshold2=120) # Canny Edge Detection

    return roi, edge_img

def createPoster(img): # private function
        
        canvas = np.zeros((400,400),dtype=np.uint8) # create poster
        canvas[75:325, 75:325] = img
        cv.rectangle(canvas,(75,75),(325,325),color=255,thickness=1, lineType=cv.LINE_AA)

        # ADD TEXT_______________________________
        Heading = 'WANTED'
        subtext1 = 'DEAD OR ALIVE'
        subtext2 = '$10'
        font_large = 4 # Scale text relative to canvas size
        font_small = font_large * 0.5  # Smaller text

        heading_pos = (80, 60)  # Top part
        subtext1_pos = (80, 360)  # Bottom part
        subtext2_pos = (160, 395)  # Near the bottom center

        canvas = cv.putText(canvas, Heading, org=heading_pos, fontFace=cv.FONT_HERSHEY_PLAIN, fontScale=font_large, color=255, thickness=1, lineType=cv.LINE_AA)
        canvas = cv.putText(canvas, subtext1, org=subtext1_pos, fontFace=cv.FONT_HERSHEY_PLAIN, fontScale=font_small, color=255, thickness=1, lineType=cv.LINE_AA)
        canvas = cv.putText(canvas, subtext2, org=subtext2_pos, fontFace=cv.FONT_HERSHEY_PLAIN, fontScale=font_small, color=255, thickness=1, lineType=cv.LINE_AA)

        return canvas

def groupEdges(img):
    img = cv.threshold(img, 127, 255, cv.THRESH_BINARY)[1] # convert to black and white - with one chanel
    num_labels, labels_img = cv.connectedComponents(img) # get the groups

    masks = []
    
    for label in range(1, num_labels):  # skip background
        mask = (labels_img == label).astype(np.uint8)
        masks.append(mask) # add group masks to array

    return num_labels, masks

def reduceNoise(img):

    # Ensure the image is in binary format (0 and 255)
    img = (img > 0).astype(np.uint8) * 255
    clean_img = np.zeros(img.shape, dtype=np.uint8)

    # Group edges using connected components
    _, masks = groupEdges(img)

    for mask in masks:
        if cv.countNonZero(mask) >= 25:
            clean_img[mask != 0] = 255
            
    # Apply Morphology kernal
    kernel = np.ones((3, 3), np.uint8)
    cleaned_img = cv.morphologyEx(clean_img, cv.MORPH_CLOSE, kernel)

    return cleaned_img

def getPaths(img):
    _, masks = groupEdges(img)

    paths = []
    for mask in masks:
        bitmap = potrace.Bitmap(mask.astype(bool))
        path = bitmap.trace()
        paths.append(path)

    return paths

In [ ]:
if __name__ == '__main__':
     
    # Test Names
    TEST1 = False
    TEST2 = True
    TEST3 = False
    POSTER = False

    Pass = False
    image = None
    score = 0

    script_dir = os.path.dirname(os.path.realpath('TESTS.ipynb'))
    img_path = os.path.join(script_dir, 'test_dataset')

    list_dir = os.listdir(img_path)
    num_imgs = len(list_dir)

    
    for i in range(num_imgs):
        # get current image
        current_path = os.path.join(img_path, f"img_{i}.jpg")
        image = cv.imread(current_path)
        
        print(f"IMAGE {i} ________________________________")

        if TEST1: # test face and edge detection accuracy
            face_img, edge_img = detectFaceEdges(image)

            image = cv.resize(image, (450, 250))
            cv.imshow(f'original image: {i}', image)

            if edge_img is None:
                cv.waitKey(0)
                cv.destroyAllWindows()
                continue
            
            # overlap face and edge image to show accuracy            
            face_img_colour = cv.cvtColor(face_img, cv.COLOR_GRAY2BGR)
            edges_colored = np.zeros_like(face_img_colour)
            face_img_colour[edge_img != 0] = [0, 0, 255]  # Red color in BGR

            print("If Main Facial Features exist then enter lowercase y")

            #cv.imshow('face_img', face_img)
            #cv.imshow('edge_img', edge_img)
            cv.imshow('overlay_img', face_img_colour)

            if cv.waitKey(0) == ord('y'):
                score += 1
                print("Features Found Confirmed")
            elif cv.waitKey(0) == ord('x'):
                cv.destroyAllWindows()
                break
            
            cv.destroyAllWindows()  

            print(f"current score = {score}")

        if TEST2: # test noise reduction
            _, edge_img = detectFaceEdges(image)

            if edge_img is None:
                continue

            clean_img = reduceNoise(edge_img)

            # Compare Number of Edges
            o_edges, _ = groupEdges(edge_img)
            c_edges, _ = groupEdges(clean_img)
            diff_cleaned = abs(c_edges-o_edges)

            print(f"Original Edges = {o_edges}")
            print(f"Cleaned Edges = {c_edges}")
            print(f"Difference = {diff_cleaned:.2f}")

            if diff_cleaned > 5:
                score += 1

            # VISUALISATION - Overaly clean image on original image
            overlay = cv.cvtColor(edge_img, cv.COLOR_GRAY2BGR)
            overlay[clean_img != 0] = [0, 0, 255] # Overlay Blue for cleaned image
            cv.imshow('Original Edge Image', edge_img)
            cv.imshow('Comparison Image', overlay)
            
            if cv.waitKey(0) == ord('x'):
                cv.destroyAllWindows()
                break
            cv.destroyAllWindows()

        if TEST3: # test path optimisation
            _, edge_img = detectFaceEdges(image)

            if edge_img is None:
                continue

            clean_img = reduceNoise(edge_img)      
            paths = getPaths(clean_img)
            num_paths, _ = groupEdges(edge_img)

            # VISUALISATION - image and graph
            #cv.imshow('Edge Image', clean_img)
            #cv.waitKey(0)
            #cv.destroyAllWindows()

            fig, ax = plt.subplots(figsize=(6, 6))

            count = 0
            for path in paths:
                for curve in path:
                    curve_verts = curve.tesselate()
                    x, y = zip(*curve_verts)
                    ax.scatter(x, y, marker='o', s=2, label=f'Group {count}')

                count += 1    
            plt.show()

            print(f"Original Number of Paths = {num_paths}")
            print(f'Reduced Number of Paths: {count}')
            if (count - num_paths) > 5:
                score += 1

        if POSTER: # Create Poster Demonstration
            _, edge_img = detectFaceEdges(image)

            if edge_img is None:
                continue

            clean_img = reduceNoise(edge_img)
            poster = createPoster(clean_img)
            
            paths = getPaths(poster)
            c_edges, _ = groupEdges(clean_img)
            num_paths = len(paths)

             # VISUALISATION - image and graph
            
            print('POSTER CREATED__________________')
            print(f"clean image edges: {c_edges}")
            print(f"poster edges: {num_paths}")

            cv.imshow('Poster', poster)
            if cv.waitKey(0) == ord('x'):
                cv.destroyAllWindows()
                break
            
            cv.destroyAllWindows()


    # Pass or Fail
    print("_______________TEST COMPLETE________________")
    print(f"Number of images in dataset: {num_imgs}")
    print(f"Number passed images: {score}")
    if (num_imgs - score) < 2:
        Pass = True
    else:
        Pass = False

    if Pass:
        print("Test Passed!")
    else:
        print("Test Failed")

IMAGE 0 ________________________________
Original Edges = 64
Cleaned Edges = 19
Difference = 45.00
POSTER CREATED__________________
clean image edges: 19
poster edges: 33
IMAGE 1 ________________________________
Original Edges = 75
Cleaned Edges = 25
Difference = 50.00
POSTER CREATED__________________
clean image edges: 25
poster edges: 39
IMAGE 2 ________________________________
No Face Detected!
_______________TEST COMPLETE________________
Number of images in dataset: 3
Number passed images: 2
Test Passed!
